In [1]:
import pandas as pd
from backtesting import Backtest, Strategy
import math
from vnstock3 import Vnstock
import talib as ta

RSI_PERIOD = 14
RSI_OVERSOLD = 30
RSI_OVERBOUGHT = 70

In [2]:
 def calculate_first_mondays(dates):
        if not isinstance(dates, pd.DatetimeIndex):
            dates = pd.DatetimeIndex(dates)
        dates_series = pd.Series(dates, index=dates)
        mondays = dates_series[dates_series.dt.dayofweek == 0]
        first_mondays = mondays.groupby([mondays.dt.year, mondays.dt.month]).first()
        return set(first_mondays)

In [3]:
class DCA(Strategy):
    average_monthly_income_vnd = 500  # Average monthly income in VND
    investment_percentage = 0.10  # Percentage of income to invest
    fund = 50  # Initialize the investment fund

    def init(self):
        close = self.data.Close
        # Calculate RSI
        self.rsi = self.I(ta.RSI, close, timeperiod=14)
        # Calculate Bollinger Bands
        self.boll_high = self.I(ta.BBANDS, close, timeperiod=15, nbdevup=2, nbdevdn=2)[0]
        self.boll_low = self.I(ta.BBANDS, close, timeperiod=15, nbdevup=2, nbdevdn=2)[2]
        self.first_mondays = calculate_first_mondays(self.data.index)
    def next(self):
        today = self.data.index[-1]
        if today in self.first_mondays:
            self.fund += self.average_monthly_income_vnd * self.investment_percentage

        # Check for buy signal
        if (self.data.Close[-1] <= self.boll_low[-1] and self.rsi[-1] <= 30):
            share_price = self.data.Close[-1]
            shares_to_buy = self.fund // share_price
            shares_to_buy = (shares_to_buy // 100) * 100
            if shares_to_buy > 0:
                self.buy(size=shares_to_buy)
                self.fund -= share_price * shares_to_buy
                print(f"Buy executed at {self.data.index[-1]} with {shares_to_buy} shares at price {share_price}, total price {share_price * shares_to_buy}")

                
def run_backtest(stock_symbol, usd_vnd_data):
    # Fetch stock data
    stock_data = Vnstock().stock(symbol=stock_symbol).quote.history(start='2019-01-01', end='2024-01-04')
    stock_data = stock_data.rename(columns={"open": "Open", "high": "High", "low": "Low", "close": "Close", "volume": "Volume"})
    stock_data.set_index('time', inplace=True)
    stock_data.index = pd.to_datetime(stock_data.index)

    # Merge USD/VND data
    stock_data.index = stock_data.index.normalize()
    stock_data['usd/vnd'] = usd_vnd_data['Close'].reindex(stock_data.index) / 1000
    stock_data['Close'] = stock_data['Close'] / stock_data['usd/vnd']
    stock_data = stock_data.dropna()

    # Run the backtest
    bt = Backtest(
        stock_data,
        DCA,
        trade_on_close=True,
    )
    stats = bt.run()
    # bt.plot(filename=f'{stock_symbol}')
    
    # Calculate investment details
    trades = stats["_trades"]
    price_paid = trades["Size"] * trades["EntryPrice"]
    total_invested = price_paid.sum()

    current_shares = trades["Size"].sum()
    current_equity = current_shares * stock_data.Close.iloc[-1]

    print(f"Results for {stock_symbol}:")
    print("Total investment:", total_invested)
    print("Current Shares:", current_shares)
    print("Current Equity:", current_equity)
    print("RoR:", ((current_equity - total_invested) / total_invested)*100)
    print("-" * 50)

# Load USD/VND data
usd_vnd_data = pd.read_csv('VND=XCommon.csv')
usd_vnd_data['Date'] = pd.to_datetime(usd_vnd_data['Date'])
usd_vnd_data.set_index('Date', inplace=True)

# List of stock symbols
stock_symbols = ['VCB', 'VPB', 'FPT', 'REE', "MSN", 'VNM']

# Run backtest for each stock
for symbol in stock_symbols:
    run_backtest(symbol, usd_vnd_data)

2024-08-14 23:34:03,928 - vnstock3.common.data.data_explorer - WARNING - Thông tin niêm yết & giao dịch sẽ được truy xuất từ TCBS


Buy executed at 2020-02-27 00:00:00 with 200.0 shares at price 2.3502581755593805, total price 470.0516351118761
Buy executed at 2020-03-02 00:00:00 with 100.0 shares at price 2.2893173797021604, total price 228.93173797021603
Buy executed at 2020-07-27 00:00:00 with 100.0 shares at price 2.1605177993527507, total price 216.05177993527508
Buy executed at 2022-04-18 00:00:00 with 300.0 shares at price 2.8730512249443207, total price 861.9153674832962
Buy executed at 2022-05-13 00:00:00 with 100.0 shares at price 2.6762209906477312, total price 267.6220990647731
Buy executed at 2022-10-03 00:00:00 with 100.0 shares at price 2.505973590442255, total price 250.5973590442255


2024-08-14 23:34:05,111 - vnstock3.common.data.data_explorer - WARNING - Thông tin niêm yết & giao dịch sẽ được truy xuất từ TCBS


Results for VCB:
Total investment: 2295.1699786096624
Current Shares: 900
Current Equity: 3126.4131551901337
RoR: 36.217063848317274
--------------------------------------------------
Buy executed at 2019-05-06 00:00:00 with 900.0 shares at price 0.2666092387207925, total price 239.94831484871327
Buy executed at 2019-11-21 00:00:00 with 1000.0 shares at price 0.28913689834963585, total price 289.1368983496358
Buy executed at 2019-12-03 00:00:00 with 200.0 shares at price 0.27017710173654497, total price 54.03542034730899
Buy executed at 2020-03-16 00:00:00 with 500.0 shares at price 0.30253404585416305, total price 151.2670229270815
Buy executed at 2021-12-23 00:00:00 with 1200.0 shares at price 0.8516663036375518, total price 1021.9995643650622
Buy executed at 2022-05-13 00:00:00 with 300.0 shares at price 0.7577069622445445, total price 227.31208867336335
Buy executed at 2022-09-26 00:00:00 with 300.0 shares at price 0.7025316455696202, total price 210.75949367088606
Buy executed at 

2024-08-14 23:34:06,529 - vnstock3.common.data.data_explorer - WARNING - Thông tin niêm yết & giao dịch sẽ được truy xuất từ TCBS


...

Results for FPT:
Total investment: 2180.980054283871
Current Shares: 1400
Current Equity: 4786.187050359712
RoR: 119.45120685348336
--------------------------------------------------
Buy executed at 2020-03-09 00:00:00 with 900.0 shares at price 0.8123599379417342, total price 731.1239441475608
Buy executed at 2021-11-22 00:00:00 with 500.0 shares at price 1.7950302334819261, total price 897.5151167409631
Buy executed at 2022-11-07 00:00:00 with 300.0 shares at price 2.019702452754322, total price 605.9107358262967
Buy executed at 2023-04-26 00:00:00 with 100.0 shares at price 2.0321220125250288, total price 203.21220125250287
Buy executed at 2023-08-18 00:00:00 with 100.0 shares at price 2.170649895178197, total price 217.0649895178197
Buy executed at 2023-10-26 00:00:00 with 100.0 shares at price 1.95114006514658, total price 195.114006514658


2024-08-14 23:34:08,655 - vnstock3.common.data.data_explorer - WARNING - Thông tin niêm yết & giao dịch sẽ được truy xuất từ TCBS


Results for REE:
Total investment: 2849.9409939998013
Current Shares: 2000
Current Equity: 4052.6207605344293
RoR: 42.2001637601172
--------------------------------------------------
Buy executed at 2019-07-18 00:00:00 with 100.0 shares at price 2.6476745440811484, total price 264.7674544081149
Buy executed at 2019-12-03 00:00:00 with 100.0 shares at price 2.238548713750162, total price 223.8548713750162
Buy executed at 2020-07-24 00:00:00 with 200.0 shares at price 1.815358067299396, total price 363.07161345987925
Buy executed at 2022-03-14 00:00:00 with 200.0 shares at price 4.921960389979452, total price 984.3920779958903
Buy executed at 2022-10-03 00:00:00 with 100.0 shares at price 3.9404736952420873, total price 394.04736952420876
Buy executed at 2023-04-25 00:00:00 with 100.0 shares at price 3.034817400187282, total price 303.4817400187282
Buy executed at 2023-09-25 00:00:00 with 100.0 shares at price 2.8858784893267653, total price 288.58784893267654


2024-08-14 23:34:09,896 - vnstock3.common.data.data_explorer - WARNING - Thông tin niêm yết & giao dịch sẽ được truy xuất từ TCBS


Results for MSN:
Total investment: 2822.2029757145137
Current Shares: 900
Current Equity: 2549.2291880781095
RoR: -9.67236552386151
--------------------------------------------------
Buy executed at 2019-08-13 00:00:00 with 100.0 shares at price 3.421381479725945, total price 342.1381479725945
Buy executed at 2020-02-03 00:00:00 with 100.0 shares at price 3.223190406349754, total price 322.3190406349754
Buy executed at 2021-01-28 00:00:00 with 100.0 shares at price 3.5822965884953835, total price 358.22965884953834
Buy executed at 2021-05-06 00:00:00 with 100.0 shares at price 3.341283607979185, total price 334.1283607979185
Buy executed at 2022-01-25 00:00:00 with 100.0 shares at price 3.1448732083792725, total price 314.48732083792726
Buy executed at 2022-05-12 00:00:00 with 100.0 shares at price 2.597830802603037, total price 259.7830802603037
Buy executed at 2023-04-17 00:00:00 with 200.0 shares at price 2.9010238907849826, total price 580.2047781569966
Buy executed at 2023-10-25 0